# ***EDA 2***

In [2]:
!pip install ppscore
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
import ppscore as pps # After installation, try importing again

# Load the dataset
file_path = '/content/sample_data/adult_with_headers.csv'
data = pd.read_csv(file_path)

# Basic data exploration
print(data.info())
print(data.describe())
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None
                age        fnlwgt  education_num  capital_gain  capital_loss  \
count  3

In [3]:
# Handle missing values (example: imputation)
data.fillna(data.median(), inplace=True)
data.fillna('Unknown', inplace=True)  # for categorical variables

<ipython-input-3-e80d9cd9ff0c>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.fillna(data.median(), inplace=True)


In [6]:
# Scaling techniques
numerical_features = data.select_dtypes(include=[np.number]).columns

# Standard Scaling
standard_scaler = StandardScaler()
data[numerical_features] = standard_scaler.fit_transform(data[numerical_features])

# Min-Max Scaling
min_max_scaler = MinMaxScaler()
data[numerical_features] = min_max_scaler.fit_transform(data[numerical_features])

In [9]:
# Encoding techniques
categorical_features = data.select_dtypes(include=[object]).columns

# One-Hot Encoding for categorical variables with less than 5 categories
one_hot_encoder = OneHotEncoder(drop='first')
for feature in categorical_features:
    if data[feature].nunique() < 5:
        encoded = one_hot_encoder.fit_transform(data[[feature]]).toarray()
        encoded_df = pd.DataFrame(encoded, columns=one_hot_encoder.get_feature_names_out([feature]))
        data = pd.concat([data, encoded_df], axis=1).drop(columns=[feature])

# Recalculate categorical features after one-hot encoding
categorical_features = data.select_dtypes(include=[object]).columns # Update categorical features

# Label Encoding for categorical variables with more than 5 categories
label_encoder = LabelEncoder()
for feature in categorical_features:
    if data[feature].nunique() >= 5:
        data[feature] = label_encoder.fit_transform(data[feature])

In [10]:
# Feature Engineering
# Example: creating new feature 'age_group'
data['age_group'] = pd.cut(data['age'], bins=[0, 25, 45, 65, 100], labels=['Young', 'Adult', 'Middle Age', 'Senior'])

# Check if 'hours-per-week' column exists before using it
if 'hours-per-week' in data.columns:
    # Example: creating new feature 'working_hours_per_week'
    data['working_hours_per_week'] = data['hours-per-week'] * 52
else:
    print("Column 'hours-per-week' not found in the DataFrame.")

# Check if 'capital-gain' column exists before transforming
if 'capital-gain' in data.columns:
    # Transforming a skewed feature (example: log transformation of 'capital-gain')
    data['capital-gain'] = np.log1p(data['capital-gain'])
else:
    print("Column 'capital-gain' not found in the DataFrame.")

# Feature Selection using Isolation Forest
# Recalculate numerical features if necessary
numerical_features = data.select_dtypes(include=[np.number]).columns
isolation_forest = IsolationForest(contamination=0.01)
outliers = isolation_forest.fit_predict(data[numerical_features])
data = data[outliers == 1]

Column 'hours-per-week' not found in the DataFrame.
Column 'capital-gain' not found in the DataFrame.


In [11]:

# Predictive Power Score (PPS) Analysis
pps_matrix = pps.matrix(data)
print(pps_matrix[pps_matrix['ppscore'] > 0.1].head(5))  # Example threshold

# Correlation matrix
correlation_matrix = data.corr()
print(correlation_matrix.head(5))

# Aligning PPS matrix with correlation matrix
pps_matrix_pivot = pps_matrix.pivot(index='x', columns='y', values='ppscore').fillna(0)
aligned_correlation_matrix = correlation_matrix.reindex(index=pps_matrix_pivot.index, columns=pps_matrix_pivot.columns).fillna(0)

# Comparing PPS with correlation matrix
comparison_df = pd.DataFrame({
    'PPS': pps_matrix_pivot.stack(),
    'Correlation': aligned_correlation_matrix.stack()
}).reset_index()

print(comparison_df.head(5))

/usr/local/lib/python3.10/dist-packages/ppscore/calculation.py:104: RuntimeWarning: invalid value encountered in scalar divide
  return f1_diff / scale_range  # 0.1/0.3 = 0.33
/usr/local/lib/python3.10/dist-packages/ppscore/calculation.py:104: RuntimeWarning: invalid value encountered in scalar divide
  return f1_diff / scale_range  # 0.1/0.3 = 0.33
/usr/local/lib/python3.10/dist-packages/ppscore/calculation.py:104: RuntimeWarning: invalid value encountered in scalar divide
  return f1_diff / scale_range  # 0.1/0.3 = 0.33
/usr/local/lib/python3.10/dist-packages/ppscore/calculation.py:104: RuntimeWarning: invalid value encountered in scalar divide
  return f1_diff / scale_range  # 0.1/0.3 = 0.33
/usr/local/lib/python3.10/dist-packages/ppscore/calculation.py:104: RuntimeWarning: invalid value encountered in scalar divide
  return f1_diff / scale_range  # 0.1/0.3 = 0.33
/usr/local/lib/python3.10/dist-packages/ppscore/calculation.py:104: RuntimeWarning: invalid value encountered in scalar 

            x               y   ppscore            case  is_valid_score  \
0         age             age  1.000000  predict_itself            True   
5         age  marital_status  0.152248      regression            True   
17  workclass       workclass  1.000000  predict_itself            True   
22  workclass      occupation  0.102331      regression            True   
34     fnlwgt          fnlwgt  1.000000  predict_itself            True   

                 metric  baseline_score  model_score                    model  
0                  None        0.000000     1.000000                     None  
5   mean absolute error        0.193200     0.163786  DecisionTreeRegressor()  
17                 None        0.000000     1.000000                     None  
22  mean absolute error        0.263743     0.236754  DecisionTreeRegressor()  
34                 None        0.000000     1.000000                     None  
                    age  workclass    fnlwgt  education  education_nu

<ipython-input-11-29ae125c0c3f>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = data.corr()
